In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
from zipfile import ZipFile
import urllib
from datetime import datetime
import string
import pygsheets
import requests
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import Bio.Entrez as Entrez
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.primers as primers
import paulssonlab.cloning.ncbi as ncbi

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Entrez

In [ ]:
# ("Escherichia coli"[Organism] AND sigma70[All Fields]) AND "Escherichia coli str. K-12 substr. MG1655"[porgn] AND alive[prop]

In [ ]:
# sigb[Gene/Protein Name] AND "Bacillus subtilis subsp. subtilis str. 168"[Organism] AND alive[prop]
# sigb[Gene/Protein Name] AND "Bacillus subtilis subsp. subtilis str. 168"[Organism] AND "srcdb refseq"[Properties] AND alive[prop]

In [ ]:
organisms = {
    "ecoli": "Escherichia coli str. K-12 substr. MG1655",
    "bsubtilis": "Bacillus subtilis subsp. subtilis str. 168",
}

In [ ]:
Entrez.esearch("genome", organisms["ecoli"]).read()

In [ ]:
Entrez.esearch("genome", "E. coli MG1655").read()

In [ ]:
Entrez.esearch("genome", organisms["bsubtilis"]).read()

In [ ]:
Entrez.esearch("genome", "B. subtilis 168").read()

In [ ]:
Entrez.esearch(
    "gene",
    'sigb[Gene/Protein Name] AND "bacillus subtilis 168"[Organism] AND "srcdb refseq"[Properties] AND alive[prop]',
).read()

In [ ]:
Entrez.esearch(
    "gene",
    'sigb[Gene/Protein Name] AND "Bacillus subtilis subsp. subtilis str. 168"[Organism] AND "srcdb refseq"[Properties] AND alive[prop]',
).read()

In [ ]:
doc = list(Entrez.parse(Entrez.efetch("gene", id="939937", retmode="xml")))[0]

In [ ]:
doc.keys()

In [ ]:
# doc['Entrezgene_comments']

In [ ]:
doc["Entrezgene_locus"][1]["Gene-commentary_products"][0][
    "Gene-commentary_genomic-coords"
]

In [ ]:
Entrez.efetch(db="nucleotide", id="939937").read()

In [ ]:
Entrez.efetch(
    db="nuccore", id="255767013", rettype="fasta", **{"from": "522862", "to": "523650"}
).read()

In [ ]:
handle = Entrez.esearch("gene", "sigB")

In [ ]:
handle.read()

In [ ]:
entrez_base_url = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils"

In [ ]:
params = {"db": "genome", "term": "E. coli"}
res = requests.get(f"{entrez_base_url}/esearch.fcgi", params=params)
res.content

In [ ]:
params = {"dbfrom": "genome", "db": "nuccore", "term": "E. coli", "retmode": "text"}
res = requests.get(f"{entrez_base_url}/elink.fcgi", params=params)
res.content

In [ ]:
a = ncbi.get_gene_seq("sigb", "bacillus subtilis 168")

In [ ]:
a

In [ ]:
a["Seq-loc_int"]["Seq-interval"]["Seq-interval_from"]

In [ ]:
a["Seq-loc_int"]["Seq-interval"]["Seq-interval_to"]

In [ ]:
a["Seq-loc_int"]["Seq-interval"]

In [ ]:
a["Gene-commentary_products"][0]

In [ ]:
ncbi.get_gene("sigma 70", "e. coli")

# BioCyc

In [ ]:
import requests

In [ ]:
requests.get("https://websvc.biocyc.org/getxml?BSUB:BSU04730")

In [ ]:
_.content